In [1]:
import pandas as pd 
df = pd.read_csv('./df.csv')
df_test = pd.read_csv('./df_test.csv')

df

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,5,5,7,10,...,7,8,7,2,2,1,4,6,4,0.495
1117953,1117953,2,2,4,3,9,5,8,1,3,...,9,4,4,3,7,4,9,4,5,0.480
1117954,1117954,7,3,9,4,6,5,9,1,3,...,5,5,5,5,6,5,5,2,4,0.485
1117955,1117955,7,3,3,7,5,2,3,4,6,...,6,8,5,3,4,6,7,6,4,0.495


In [2]:
df_test

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,...,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,...,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,...,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,...,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,...,4,3,2,6,4,6,8,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
745301,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
745302,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [3]:
# 删除重复
df=df.drop_duplicates()
df

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,5,5,7,10,...,7,8,7,2,2,1,4,6,4,0.495
1117953,1117953,2,2,4,3,9,5,8,1,3,...,9,4,4,3,7,4,9,4,5,0.480
1117954,1117954,7,3,9,4,6,5,9,1,3,...,5,5,5,5,6,5,5,2,4,0.485
1117955,1117955,7,3,3,7,5,2,3,4,6,...,6,8,5,3,4,6,7,6,4,0.495


In [4]:
df_test # 测试集数量  745305

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,...,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,...,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,...,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,...,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,...,4,3,2,6,4,6,8,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
745301,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
745302,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


# 特征处理

In [7]:
train_x,train_y,test_x,n_jobs = df.drop('FloodProbability',axis=1),df['FloodProbability'],df_test,8

In [8]:
from openfe import OpenFE, transform

ofe = OpenFE()
features = ofe.fit(data=train_x, label=train_y, n_jobs=n_jobs)  # generate new features
train_x, test_x = transform(train_x, test_x, features, n_jobs=n_jobs) # transform the train and test data according to generated features.

The number of candidate features is 4608
Start stage I selection.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [24:12<00:00, 45.40s/it]


777 same features have been deleted.
Meet early-stopping in successive feature-wise halving.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [56:51<00:00, 106.60s/it]


The number of remaining candidate features is 2000
Start stage II selection.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [33:12<00:00, 62.28s/it]


MemoryError: Unable to allocate 16.7 GiB for an array with shape (2000, 1117957) and data type float64

In [5]:
df_solve = pd.concat([df.drop('FloodProbability',axis=1).copy(),df_test],axis=0)
df_solve = df_solve.reset_index(drop=True)
df_solve

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [6]:
# 添加缺失特征
from feature_engine.imputation import AddMissingIndicator

ami = AddMissingIndicator()
ami.fit(df_solve)
df_solve_mi = ami.transform(df_solve)
df_solve_mi

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [7]:
#频率编码


df_solve_mi_cf = df_solve_mi.copy()
df_solve_mi_cf

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [8]:
#分类空白插值
from feature_engine.imputation import MeanMedianImputer
mmi = MeanMedianImputer(imputation_method='median')
mmi.fit(df_solve_mi_cf)
df_solve_mi_cf_mmi = mmi.transform(df_solve_mi_cf)

df_solve_mi_cf_mmi

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [9]:
#特征扩展
from feature_engine.creation import CyclicalFeatures
cf = CyclicalFeatures()
cf.fit(df_solve_mi_cf_mmi)
df_solve_mi_cf_mmi_v1 = cf.transform(df_solve_mi_cf_mmi)
df_solve_mi_cf_mmi_v1

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin,PoliticalFactors_cos
0,0,5,8,5,8,6,4,4,3,3,...,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,0.923880,3.826834e-01
1,1,6,7,4,4,8,8,3,5,4,...,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,0.923880,3.826834e-01
2,2,6,5,6,7,3,7,1,5,4,...,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,0.923880,3.826834e-01
3,3,3,4,6,5,4,8,4,7,6,...,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,0.923880,-3.826834e-01
4,4,5,3,2,6,4,4,3,3,3,...,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,0.923880,-3.826834e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,0.798017,-0.602635,0.969400,0.245485,0.909632,0.415415,0.707107,-7.071068e-01,0.707107,-7.071068e-01
1863258,1863258,4,4,2,12,4,3,4,3,5,...,0.961826,-0.273663,0.996584,-0.082579,0.755750,0.654861,0.923880,-3.826834e-01,1.000000,6.123234e-17
1863259,1863259,5,7,9,5,5,6,7,5,5,...,0.961826,-0.273663,0.164595,-0.986361,0.989821,0.142315,0.923880,-3.826834e-01,1.000000,6.123234e-17
1863260,1863260,4,7,6,3,5,2,3,8,6,...,0.895163,0.445738,0.475947,-0.879474,0.909632,-0.415415,0.923880,-3.826834e-01,0.923880,-3.826834e-01


In [10]:
from feature_engine.selection import SmartCorrelatedSelection

scs = SmartCorrelatedSelection(threshold=0.7)
df_solve_mi_cf_mmi_select = scs.fit_transform(df_solve_mi_cf_mmi_v1)
df_solve_mi_cf_mmi_select

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,PoliticalFactors,id_cos,MonsoonIntensity_sin,TopographyDrainage_sin,RiverManagement_sin,...,Watersheds_cos,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin
0,0,5,8,5,8,3,1.0,0.923880,0.342020,9.238795e-01,...,-3.826834e-01,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,0.923880
1,1,6,7,4,4,3,1.0,0.707107,0.642788,1.000000e+00,...,3.826834e-01,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,0.923880
2,2,6,5,6,7,3,1.0,0.707107,0.984808,7.071068e-01,...,-3.826834e-01,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,0.923880
3,3,3,4,6,5,5,1.0,0.923880,0.984808,7.071068e-01,...,6.123234e-17,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,0.923880
4,4,5,3,2,6,5,1.0,0.923880,0.866025,7.071068e-01,...,-7.071068e-01,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,0.923880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,6,1.0,0.923880,0.984808,1.224647e-16,...,-3.826834e-01,0.798017,-0.602635,0.969400,0.245485,0.909632,0.415415,0.707107,-7.071068e-01,0.707107
1863258,1863258,4,4,2,12,4,1.0,1.000000,0.984808,7.071068e-01,...,3.826834e-01,0.961826,-0.273663,0.996584,-0.082579,0.755750,0.654861,0.923880,-3.826834e-01,1.000000
1863259,1863259,5,7,9,5,4,1.0,0.923880,0.642788,-3.826834e-01,...,6.123234e-17,0.961826,-0.273663,0.164595,-0.986361,0.989821,0.142315,0.923880,-3.826834e-01,1.000000
1863260,1863260,4,7,6,3,5,1.0,1.000000,0.642788,7.071068e-01,...,7.071068e-01,0.895163,0.445738,0.475947,-0.879474,0.909632,-0.415415,0.923880,-3.826834e-01,0.923880


In [11]:
df_solve_mi_cf_mmi_select_cp1 = df_solve_mi_cf_mmi_select.copy()
df_solve_mi_cf_mmi_select = df_solve_mi_cf_mmi_select[:1117957].copy()
df_solve_mi_cf_mmi_select['FloodProbability'] = df[:1117957]['FloodProbability']
# df_solve_mi_cf_mmi_select.reset_index(drop=True, inplace=True)
df_solve_mi_cf_mmi_select

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,PoliticalFactors,id_cos,MonsoonIntensity_sin,TopographyDrainage_sin,RiverManagement_sin,...,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin,FloodProbability
0,0,5,8,5,8,3,1.000000,0.923880,0.342020,0.923880,...,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,9.238795e-01,0.445
1,1,6,7,4,4,3,1.000000,0.707107,0.642788,1.000000,...,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,9.238795e-01,0.450
2,2,6,5,6,7,3,1.000000,0.707107,0.984808,0.707107,...,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,9.238795e-01,0.530
3,3,3,4,6,5,5,1.000000,0.923880,0.984808,0.707107,...,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,9.238795e-01,0.535
4,4,5,3,2,6,5,1.000000,0.923880,0.866025,0.707107,...,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,9.238795e-01,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,-0.809026,0.923880,0.866025,1.000000,...,0.673696,0.739009,0.324699,0.945817,0.909632,0.415415,0.707107,-7.071068e-01,1.000000e+00,0.495
1117953,1117953,2,2,4,3,5,-0.809024,0.707107,0.642788,1.000000,...,0.526432,-0.850217,0.969400,0.245485,0.540641,-0.841254,1.000000,6.123234e-17,9.238795e-01,0.480
1117954,1117954,7,3,9,4,4,-0.809022,0.382683,0.866025,-0.382683,...,0.798017,-0.602635,0.996584,-0.082579,0.989821,0.142315,0.707107,7.071068e-01,1.000000e+00,0.485
1117955,1117955,7,3,3,7,4,-0.809020,0.382683,0.866025,0.923880,...,0.995734,0.092268,0.915773,-0.401695,0.909632,-0.415415,0.707107,-7.071068e-01,1.000000e+00,0.495


In [25]:
df

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,5,5,7,10,...,7,8,7,2,2,1,4,6,4,0.495
1117953,1117953,2,2,4,3,9,5,8,1,3,...,9,4,4,3,7,4,9,4,5,0.480
1117954,1117954,7,3,9,4,6,5,9,1,3,...,5,5,5,5,6,5,5,2,4,0.485
1117955,1117955,7,3,3,7,5,2,3,4,6,...,6,8,5,3,4,6,7,6,4,0.495


# 模型训练

In [26]:
df_test

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,1117957,4,6,3,5,6,7,8,7,8,...,5,7,5,6,3,6,4,4,5,0.475371
1,1117958,4,4,2,9,5,5,4,7,5,...,4,7,4,5,1,7,4,4,3,0.472348
2,1117959,1,3,6,5,7,2,4,6,4,...,9,2,5,5,2,3,6,8,3,0.505486
3,1117960,2,4,4,6,4,5,4,3,4,...,8,4,6,7,6,4,2,4,4,0.539111
4,1117961,6,3,2,4,6,4,5,5,3,...,3,2,6,4,6,8,4,5,5,0.397402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,6,1,3,5,6,4,4,6,6,0.606393
745301,1863258,4,4,2,12,4,3,4,3,5,...,7,4,4,3,5,5,3,5,4,0.475305
745302,1863259,5,7,9,5,5,6,7,5,5,...,11,3,11,4,5,9,5,5,4,0.572261
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,8,6,2,3,8,7,5,5,0.476002


In [ ]:
train_x['FloodProbability'] = df['FloodProbability']
train_x

In [ ]:
from autogluon.tabular import TabularPredictor
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

custom_hyperparameters = get_hyperparameter_config('default')

def get_oof_ts(predictor,test_x):
    predictions_oof = predictor.predict_oof()
    prdict_y = predictor.predict(test_x)
    oof = pd.DataFrame({'oofs': predictions_oof, 'predict_y': prdict_y}).to_csv('model{}.csv',index=False)
    return predictions_oof,prdict_y
# 训练集原始为 117957 条数据
# 初始训练
# df_train = df # df_solve_mi_cf_mmi_select[:1117957]
# df_test2 =  df_test  # df_solve_mi_cf_mmi_select  #[:2000]


# predictor = TabularPredictor(label="FloodProbability").fit(
#     train_x,#df_train,
#     presets= 'best_quality', # ['optimize_for_deployment'],  #'best_quality',
# #     time_limit=60,
#     # num_stack_levels=1,
#     # num_bag_folds=3,
# #     time_limit=3600*15
# )
# predictions = predictor.predict(df_test2)

predictor = TabularPredictor(
    label="FloodProbability",
    problem_type='regression',
    eval_metric='r2'
).fit(
    train_x, #df, #df_train,
    presets='best_quality',
#     unlabeled_data = df_test2,   #df_test2.drop(columns='Survived'), #df_train[800:].drop(columns='Survived'),
    # auto_stack=True,
    feature_generator=auto_ml_pipeline_feature_generator,
    ag_args_ensemble = {'fold_fitting_strategy':'sequential_local'},
    # dynamic_stacking=True,
#     num_stack_levels=1,
    num_bag_folds=10,
    # time_limit=60*4,
#     included_model_types = ['XGB','KNN',''],
#     excluded_model_types = ['NN_TORCH'],
    # use_bag_holdout=True,
    # hyperparameters=custom_hyperparameters,
    # hyperparameter_tune_kwargs='auto',#hyperparameter_tune_kwargs,
    keep_only_best = True,
)
aa,predictions = get_oof_ts(predictor,test_x)

In [ ]:
predictions

In [ ]:
predictor.leaderboard()

In [14]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(predictor, df_train.drop('FloodProbability',axis=1), df_train['FloodProbability'], scoring = 'accuracy', cv = 10).mean()

In [15]:
# predictor.evaluate(df_test2)


KeyboardInterrupt



In [16]:
df_test

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,...,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,...,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,...,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,...,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,...,4,3,2,6,4,6,8,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
745301,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
745302,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [17]:
df_train['FloodProbability']

0          0.445
1          0.450
2          0.530
3          0.535
4          0.415
           ...  
1117952    0.495
1117953    0.480
1117954    0.485
1117955    0.495
1117956    0.560
Name: FloodProbability, Length: 1117957, dtype: float64

In [18]:
pl = predictions
# pl = list(int(i) for i in predictions)
pl

0          0.475371
1          0.472348
2          0.505486
3          0.539111
4          0.397402
             ...   
1117952    0.498129
1117953    0.485137
1117954    0.495914
1117955    0.510269
1117956    0.570967
Name: FloodProbability, Length: 1117957, dtype: float32

In [19]:
df_test['FloodProbability'] = pl
df_test
df_sub = df_test[['id','FloodProbability']]
df_sub

,id,FloodProbability
0,1117957,0.475371
1,1117958,0.472348
2,1117959,0.505486
3,1117960,0.539111
4,1117961,0.397402
...,...,...
745300,1863257,0.606393
745301,1863258,0.475305
745302,1863259,0.572261
745303,1863260,0.476002


In [20]:
df_sub.to_csv('submission.csv',index=False)

In [21]:
cc = df_sub['FloodProbability'].value_counts()
cc

FloodProbability
0.550441    7
0.522796    7
0.522385    6
0.522915    6
0.485179    6
           ..
0.471643    1
0.472415    1
0.562462    1
0.660151    1
0.476002    1
Name: count, Length: 656643, dtype: int64

In [22]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "Message"

Successfully submitted to Regression with a Flood Prediction Dataset



  0%|          | 0.00/13.9M [00:00<?, ?B/s]
  0%|          | 8.00k/13.9M [00:00<11:48, 20.5kB/s]
 32%|███▏      | 4.50M/13.9M [00:00<00:01, 7.44MB/s]
 53%|█████▎    | 7.30M/13.9M [00:00<00:00, 9.80MB/s]
 75%|███████▍  | 10.4M/13.9M [00:01<00:00, 12.0MB/s]
 90%|████████▉ | 12.4M/13.9M [00:01<00:00, 11.9MB/s]
100%|██████████| 13.9M/13.9M [00:04<00:00, 3.40MB/s]
